# Translation of short phrases using an intermediate language:
## A thesis on leveraging interlingual approaches

## Libraries

In [39]:
import numpy as np
import pandas as pd
import itertools as it

from gensim.models import KeyedVectors

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu #pip install nltk

In [56]:
ref = ["estou fazendo um teste".split()]
frase = "fazendo um teste".split()
#cc = SmoothingFunction()
sentence_bleu(ref, frase, weights=(0, 1, 0), smoothing_function=cc.method4)
#frase

0.7165313105737893

## 1 - Loading data
Loading the models and sentences used.
- Models: https://fasttext.cc/docs/en/crawl-vectors.html
- Sentences: https://github.com/alexa/massive

In [42]:
def load_files(model_path, sentences_path, limit = None):
    '''
    Load models from FastText folder and sentences from Amazon Massive folder.
    
    Params:
    - model_path: path to the folder containing all models used, i.e., FastText
    - sentences_path: path to the folder containing all sentences used, i.e., Amazon_Massive
    - limit: define a limit in case your have low computer power, e.g., 5000
    
    Return:
    Tuple containing the language model and its corresponding sentences
    '''

    model = KeyedVectors.load_word2vec_format(model_path, unicode_errors = 'replace', limit = limit)
    sentences = pd.read_json(sentences_path, lines = True)['utt']
    
    return model, sentences

Defining data path.

In [43]:
FASTTEXT_PATH = 'Datasets/FastText/'
MASSIVE_PATH = 'Datasets/Amazon_Massive/'

In [44]:
PATHS = {
    'de': [ FASTTEXT_PATH + 'cc.de.300.vec', MASSIVE_PATH + 'de-DE.jsonl' ],
    'en': [ FASTTEXT_PATH + 'cc.en.300.vec', MASSIVE_PATH + 'en-US.jsonl' ],
    'es': [ FASTTEXT_PATH + 'cc.es.300.vec', MASSIVE_PATH + 'es-ES.jsonl' ],
    'it': [ FASTTEXT_PATH + 'cc.it.300.vec', MASSIVE_PATH + 'it-IT.jsonl' ],
    'pt': [ FASTTEXT_PATH + 'cc.pt.300.vec', MASSIVE_PATH + 'pt-PT.jsonl' ],
    'sv': [ FASTTEXT_PATH + 'cc.sv.300.vec', MASSIVE_PATH + 'sv-SE.jsonl' ],
}

LANGUAGES = PATHS.keys()

**Note**: the cell below takes approximately 5 to 7 minutes per model, if no limit is set.

In [45]:
MODELS, SENTENCES = {}, {}

for language, value in PATHS.items():
    model = value[0]
    sentences = value[1]

    print(f'Loading {model}...')
    MODELS[language], SENTENCES[language] = load_files(model, sentences, 600_000)
    print(f'Finished loading {model}\n')

print('\nAll models and sentences are now loaded!')

Loading Datasets/FastText/cc.de.300.vec...
Finished loading Datasets/FastText/cc.de.300.vec

Loading Datasets/FastText/cc.en.300.vec...
Finished loading Datasets/FastText/cc.en.300.vec

Loading Datasets/FastText/cc.es.300.vec...
Finished loading Datasets/FastText/cc.es.300.vec

Loading Datasets/FastText/cc.it.300.vec...
Finished loading Datasets/FastText/cc.it.300.vec

Loading Datasets/FastText/cc.pt.300.vec...
Finished loading Datasets/FastText/cc.pt.300.vec

Loading Datasets/FastText/cc.sv.300.vec...
Finished loading Datasets/FastText/cc.sv.300.vec


All models and sentences are now loaded!


## 2 - Preparing data

In [46]:
SAMPLES = { key: [] for key in LANGUAGES }

In [47]:
# Since all sentence files have the same length, we chose one at random for the range function.
# We prove this in the cell below
for idx in range(len(SENTENCES['pt'])):
    
    actual_sentence = { key: [] for key in LANGUAGES }
    
    try:
        for lang, sent in SENTENCES.items():
            for word in sent[idx].split(' '):
                actual_sentence[lang].append(MODELS[lang][word])

    except KeyError:
        continue
    
    for key, value in actual_sentence.items():
        SAMPLES[key].append([SENTENCES[key][idx], sum(value)])

In [48]:
for key in SENTENCES:
    SIZE_SAMPLES = len(SAMPLES[key])
    SIZE_SENTENCES = len(SENTENCES[key])
    print(
        f'Total sentences in { key } file: { SIZE_SENTENCES }'
        f' -> Model { key } samples: { SIZE_SAMPLES } ({ SIZE_SAMPLES / SIZE_SENTENCES * 100:.2f}%)'
    )

Total sentences in de file: 16521 -> Model de samples: 8117 (49.13%)
Total sentences in en file: 16521 -> Model en samples: 8117 (49.13%)
Total sentences in es file: 16521 -> Model es samples: 8117 (49.13%)
Total sentences in it file: 16521 -> Model it samples: 8117 (49.13%)
Total sentences in pt file: 16521 -> Model pt samples: 8117 (49.13%)
Total sentences in sv file: 16521 -> Model sv samples: 8117 (49.13%)


Splitting into train and test.

In [49]:
SPLIT_RATE = int(SIZE_SAMPLES * 0.7)

TRAIN_SET = { key: SAMPLES[key][:SPLIT_RATE] for key in LANGUAGES }
TEST_SET = { key: SAMPLES[key][SPLIT_RATE:] for key in LANGUAGES }

## 3 - Evaluating Control Group

Theorically speaking, translating the vector that one sentence represents to another should result in a similar sentence. For that purpose, we evaluate our results using the cosine similarity, which range is from -1 to 1.

In [50]:
TRANSLATIONS = { key: { lang: None for lang in LANGUAGES if lang != key } for key in LANGUAGES }

for origin, target in it.permutations(LANGUAGES, 2): 

    samples_origin = [sample[1] for sample in TRAIN_SET[origin]]
    samples_target = [sample[1] for sample in TRAIN_SET[target]]

    U, Sig, Vt = np.linalg.svd(np.transpose(samples_origin) @ samples_target)
    
    TRANSLATOR = np.transpose(Vt) @ np.transpose(U)
    TRANSLATIONS[origin][target] = TRANSLATOR

In [53]:
def evaluate_single_cosine_similarity(origin_lang, target_lang):
    '''
    Evaluate cosine similarity between single sentences.
    Cosine similarity has an interval from -1 to 1, and the closer to 1 the value is, more similar the params are.

    Params:
    - origin_lang: language in which the words in word_list are written
    - target_lang: language you wish to know the translation

    Example of usage:
    evaluate_single_cosine_similarity('pt', 'en')
    '''
    cc = SmoothingFunction()
    for index in range(10):
        print('Original sentence:', TEST_SET[origin_lang][index][0])
        print('Target sentence:', TEST_SET[target_lang][index][0])

        vector_translated = TRANSLATIONS[origin_lang][target_lang] @ TEST_SET[origin_lang][index][1]
        vector_target = TEST_SET[target_lang][index][1]
        
        similarities = [(s, cosine_similarity([vector_translated], [v])) for s, v in SAMPLES[target_lang]]
        most_similar = max(similarities, key= lambda x: x[1])
        print('Generated sentence:', most_similar[0])
        
        BLEU_score = sentence_bleu([TEST_SET[target_lang][index][0].split()], most_similar[0].split(), weights=(1, 0, 0, 0), smoothing_function=cc.method4)
        print('BLEU score:', BLEU_score)
        
        print('Cossine similarity to generated:', most_similar[1][0][0])
        print("Cossine similarity to target:", cosine_similarity([vector_translated], [vector_target])[0][0], "\n")

### Portuguese -> Spanish

In [54]:
evaluate_single_cosine_similarity('pt', 'es')

Original sentence: diz-me as listas disponíveis
Target sentence: nombrar las listas disponibles
Generated sentence: muéstrame las listas disponibles
BLEU score: 0.75
Cossine similarity to generated: 0.888038
Cossine similarity to target: 0.8580996 

Original sentence: adiciona lembrar de ligar à mãe aos meus lembretes
Target sentence: añade recordatorio para llamar a mamá en mis recordatorios
Generated sentence: añadir la canción a mis favoritos
BLEU score: 0.2021768865708778
Cossine similarity to generated: 0.83262116
Cossine similarity to target: 0.819102 

Original sentence: podes ler-me as minhas listas
Target sentence: puedes leerme mis listas
Generated sentence: puedes decirme las últimas noticias
BLEU score: 0.2
Cossine similarity to generated: 0.87350416
Cossine similarity to target: 0.76154476 

Original sentence: ei quias são as minhas listas
Target sentence: eh qué hay en mis listas
Generated sentence: cuales son las noticias ultimamente
BLEU score: 0
Cossine similarity to g

### English -> German

In [55]:
evaluate_single_cosine_similarity('en', 'de')

Original sentence: name the lists available
Target sentence: nenne die verfügbaren listen
Generated sentence: die verfügbaren listen
BLEU score: 0.7165313105737893
Cossine similarity to generated: 0.7748393
Cossine similarity to target: 0.7640162 

Original sentence: add remember to call mom to my reminders
Target sentence: füge denke daran mutter anzurufen zu meinen erinnerungen hinzu
Generated sentence: füge denke daran mutter anzurufen zu meinen erinnerungen hinzu
BLEU score: 1.0
Cossine similarity to generated: 0.9019309
Cossine similarity to target: 0.9019309 

Original sentence: can you read me my lists
Target sentence: kannst du meine listen vorlesen
Generated sentence: ich möchte meiner familie eine email schicken hilfst du mir
BLEU score: 0.10000000000000002
Cossine similarity to generated: 0.90858644
Cossine similarity to target: 0.7916864 

Original sentence: hey what are my lists
Target sentence: hey was sind meine listen
Generated sentence: hey was sind meine listen
BLEU s

### Avaliating path
We use the following metrics for that purpose:
- Cosine similarity
- Euclidean distance

In [29]:
def pairwise(iterable):
    '''
    Return successive overlapping pairs taken from the input iterable.
    The number of 2-tuples in the output iterator will be one fewer than the number of inputs. 
    It will be empty if the input iterable has fewer than two values.
    pairwise('ABCDEFG') --> AB BC CD DE EF FG

    Source: https://docs.python.org/3/library/itertools.html#itertools.pairwise
    '''
    a, b = it.tee(iterable)
    next(b, None)
    return zip(a, b)

In [30]:
def avaliate_path(path):
    '''
    Avaliate the translation path using cosine similarity, euclidean distance and manhattan distance.

    Params:
    - path: path of desired translation
    
    Return:
    - Score of each avaliation method
    
    Example of usage:
    avaliate_path(['pt', 'en', 'es'])
    '''
    
    translation_matrix = np.identity(300)

    for (origin, target) in pairwise(path):
        translation_matrix = TRANSLATIONS[origin][target] @ translation_matrix
        evaluate_single_cosine_similarity(origin, target)
    
    vectors = [translation_matrix @ v for _, v in TEST_SET[path[0]] ]
    vectors_target = [v for _, v in TEST_SET[path[-1]]]
    
    mean_cos_sim = sum([cosine_similarity([v1], [v2]) for v1, v2 in zip(vectors, vectors_target)])/ len(vectors)
    mean_euc_dist = sum([euclidean_distances([v1], [v2]) for v1, v2 in zip(vectors, vectors_target)])/ len(vectors)
    
    return mean_cos_sim[0][0], mean_euc_dist[0][0]

In [31]:
def avaliate_possible_paths(languages):
    '''
    Avaliate possible paths from first to last language on the list, changing the languages in the middle
    
    Params:
    - languages: list of languages
    
    Return:
    - Dataframe with the score of each path on cossine similarity, euclidean distance and manhattan distance
    
    Example of usage:
    avaliate_paths(['pt', 'en', 'es'])
    '''
    
    start = languages[0]
    end = languages[-1]
    
    paths = [[start, end]]
    for i in range(len(languages) - 2):
        for comb in it.combinations(languages[1: -1], i + 1):
            paths.append([start] + list(comb) + [end])
    
    scores = [avaliate_path(p) for p in paths]
    index = [ ' -> '.join(p) for p in paths]

    return pd.DataFrame(data=scores, columns=['Cosine Similarity', 'Euclidean Distance'], index = index)


## 4 - Study Cases

### Study Case #1: Portuguese - Italian - Spanish
In this case, we intend to evaluate how good is a translation between two languages from the Romance language family, such as Portuguese and Spanish, and if adding a language from the same family, such as Italian, affects the quality of the translation.

In [32]:
exp1 = avaliate_possible_paths(['pt', 'it', 'es'])
exp1

Original sentence: diz-me as listas disponíveis
Target sentence: nombrar las listas disponibles
Generated sentence: muéstrame las listas disponibles
Cossine similarity to generated: 0.888038
Cossine similarity to target: 0.8580996 

Original sentence: adiciona lembrar de ligar à mãe aos meus lembretes
Target sentence: añade recordatorio para llamar a mamá en mis recordatorios
Generated sentence: añadir la canción a mis favoritos
Cossine similarity to generated: 0.83262116
Cossine similarity to target: 0.819102 

Original sentence: podes ler-me as minhas listas
Target sentence: puedes leerme mis listas
Generated sentence: puedes decirme las últimas noticias
Cossine similarity to generated: 0.87350416
Cossine similarity to target: 0.76154476 

Original sentence: ei quias são as minhas listas
Target sentence: eh qué hay en mis listas
Generated sentence: cuales son las noticias ultimamente
Cossine similarity to generated: 0.77319026
Cossine similarity to target: 0.61282545 

Original sente

,Cosine Similarity,Euclidean Distance
pt -> es,0.757849,3.928147
pt -> it -> es,0.745219,4.014195


### Study Case #2: English - Swedish - German
In this case, we intend to evaluate how good is a translation between two languages from the Anglo-Saxon language family, such as English and German, and if adding a language from the same family, such as Swedish, affects the quality of the translation.

In [33]:
exp2 = avaliate_possible_paths(['en', 'sv', 'de'])
exp2

Original sentence: name the lists available
Target sentence: nenne die verfügbaren listen
Generated sentence: die verfügbaren listen
Cossine similarity to generated: 0.7748393
Cossine similarity to target: 0.7640162 

Original sentence: add remember to call mom to my reminders
Target sentence: füge denke daran mutter anzurufen zu meinen erinnerungen hinzu
Generated sentence: füge denke daran mutter anzurufen zu meinen erinnerungen hinzu
Cossine similarity to generated: 0.9019309
Cossine similarity to target: 0.9019309 

Original sentence: can you read me my lists
Target sentence: kannst du meine listen vorlesen
Generated sentence: ich möchte meiner familie eine email schicken hilfst du mir
Cossine similarity to generated: 0.90858644
Cossine similarity to target: 0.7916864 

Original sentence: hey what are my lists
Target sentence: hey was sind meine listen
Generated sentence: hey was sind meine listen
Cossine similarity to generated: 0.8924559
Cossine similarity to target: 0.8924559 



,Cosine Similarity,Euclidean Distance
en -> de,0.748613,4.646439
en -> sv -> de,0.734172,4.742345


### Study Case #3: Portuguese - English - Spanish
In this case, we intend to evaluate how good is a translation between two languages from the Romance language family, such as Portuguese and Spanish, and if adding a language from an outer group, such as English from the Anglo-Saxon, affects the quality of the translation.

In [19]:
exp3 = avaliate_possible_paths(['pt', 'en', 'es'])
exp3

Original sentence: diz-me as listas disponíveis
Target sentence: nombrar las listas disponibles
Generated sentence: muéstrame las listas disponibles
Cossine similarity to generated: 0.888038
Cossine similarity to target: 0.8580996 

Original sentence: adiciona lembrar de ligar à mãe aos meus lembretes
Target sentence: añade recordatorio para llamar a mamá en mis recordatorios
Generated sentence: añadir la canción a mis favoritos
Cossine similarity to generated: 0.83262116
Cossine similarity to target: 0.819102 

Original sentence: podes ler-me as minhas listas
Target sentence: puedes leerme mis listas
Generated sentence: puedes decirme las últimas noticias
Cossine similarity to generated: 0.87350416
Cossine similarity to target: 0.76154476 

Original sentence: ei quias são as minhas listas
Target sentence: eh qué hay en mis listas
Generated sentence: cuales son las noticias ultimamente
Cossine similarity to generated: 0.77319026
Cossine similarity to target: 0.61282545 

Original sente

,Cosine Similarity,Euclidean Distance
pt -> es,0.757849,3.928147
pt -> en -> es,0.727482,4.177093


### Study Case #4: English - Portuguese - German
In this case, we intend to evaluate how good is a translation between two languages from the Anglo-Saxon language family, such as English and German, and if adding a language from an outer group, such as Portuguese from the Romance, affects the quality of the translation.

In [20]:
exp4 = avaliate_possible_paths(['pt', 'en', 'es'])
exp4

Original sentence: diz-me as listas disponíveis
Target sentence: nombrar las listas disponibles
Generated sentence: muéstrame las listas disponibles
Cossine similarity to generated: 0.888038
Cossine similarity to target: 0.8580996 

Original sentence: adiciona lembrar de ligar à mãe aos meus lembretes
Target sentence: añade recordatorio para llamar a mamá en mis recordatorios
Generated sentence: añadir la canción a mis favoritos
Cossine similarity to generated: 0.83262116
Cossine similarity to target: 0.819102 

Original sentence: podes ler-me as minhas listas
Target sentence: puedes leerme mis listas
Generated sentence: puedes decirme las últimas noticias
Cossine similarity to generated: 0.87350416
Cossine similarity to target: 0.76154476 

Original sentence: ei quias são as minhas listas
Target sentence: eh qué hay en mis listas
Generated sentence: cuales son las noticias ultimamente
Cossine similarity to generated: 0.77319026
Cossine similarity to target: 0.61282545 

Original sente

,Cosine Similarity,Euclidean Distance
pt -> es,0.757849,3.928147
pt -> en -> es,0.727482,4.177093


## Experiments

### Experiment #1: Portuguese - Spanish - German
In this experiment, we intend to evaluate translations using languages from different groups.

In [21]:
exp5 = avaliate_possible_paths(['pt', 'es', 'de'])
exp5

Original sentence: diz-me as listas disponíveis
Target sentence: nenne die verfügbaren listen
Generated sentence: die verfügbaren listen
Cossine similarity to generated: 0.72507477
Cossine similarity to target: 0.6816883 

Original sentence: adiciona lembrar de ligar à mãe aos meus lembretes
Target sentence: füge denke daran mutter anzurufen zu meinen erinnerungen hinzu
Generated sentence: füge denke daran mutter anzurufen zu meinen erinnerungen hinzu
Cossine similarity to generated: 0.7907492
Cossine similarity to target: 0.7907492 

Original sentence: podes ler-me as minhas listas
Target sentence: kannst du meine listen vorlesen
Generated sentence: liste die spiele die du mit mir spielen kannst auf
Cossine similarity to generated: 0.72701496
Cossine similarity to target: 0.62487876 

Original sentence: ei quias são as minhas listas
Target sentence: hey was sind meine listen
Generated sentence: hey was sind meine listen
Cossine similarity to generated: 0.6982921
Cossine similarity to 

,Cosine Similarity,Euclidean Distance
pt -> de,0.662475,4.462124
pt -> es -> de,0.656088,4.493955


### Experiment #2: Portuguese - Swedish - German 

In [22]:
exp6 = avaliate_possible_paths(['pt', 'sv', 'de'])
exp6

Original sentence: diz-me as listas disponíveis
Target sentence: nenne die verfügbaren listen
Generated sentence: die verfügbaren listen
Cossine similarity to generated: 0.72507477
Cossine similarity to target: 0.6816883 

Original sentence: adiciona lembrar de ligar à mãe aos meus lembretes
Target sentence: füge denke daran mutter anzurufen zu meinen erinnerungen hinzu
Generated sentence: füge denke daran mutter anzurufen zu meinen erinnerungen hinzu
Cossine similarity to generated: 0.7907492
Cossine similarity to target: 0.7907492 

Original sentence: podes ler-me as minhas listas
Target sentence: kannst du meine listen vorlesen
Generated sentence: liste die spiele die du mit mir spielen kannst auf
Cossine similarity to generated: 0.72701496
Cossine similarity to target: 0.62487876 

Original sentence: ei quias são as minhas listas
Target sentence: hey was sind meine listen
Generated sentence: hey was sind meine listen
Cossine similarity to generated: 0.6982921
Cossine similarity to 

,Cosine Similarity,Euclidean Distance
pt -> de,0.662475,4.462124
pt -> sv -> de,0.647281,4.550113


### Experiment #3: English - Swedish - Italian

In [23]:
exp7 = avaliate_possible_paths(['en', 'sv', 'it'])
exp7

Original sentence: name the lists available
Target sentence: enuncia le liste disponibili
Generated sentence: rimuovi la lista dal database
Cossine similarity to generated: 0.6921283
Cossine similarity to target: 0.5678471 

Original sentence: add remember to call mom to my reminders
Target sentence: aggiungi ricorda di chiamare mamma ai miei promemoria
Generated sentence: rimuovi spesa al supermercato dalla mia lista delle cose da fare domenica
Cossine similarity to generated: 0.85635823
Cossine similarity to target: 0.763936 

Original sentence: can you read me my lists
Target sentence: puoi leggermi le mie liste
Generated sentence: puoi cucinare qualcosa di speciale per me e mia moglie
Cossine similarity to generated: 0.8514999
Cossine similarity to target: 0.7323967 

Original sentence: hey what are my lists
Target sentence: ciao quali sono le mie liste
Generated sentence: dimmi quali sono i film migliori nella mia zona
Cossine similarity to generated: 0.83229375
Cossine similarity

,Cosine Similarity,Euclidean Distance
en -> it,0.707053,5.180112
en -> sv -> it,0.688648,5.316212


### Experiment #4: English - Spanish - Italian

In [24]:
exp8 = avaliate_possible_paths(['en', 'es', 'it'])
exp8

Original sentence: name the lists available
Target sentence: enuncia le liste disponibili
Generated sentence: rimuovi la lista dal database
Cossine similarity to generated: 0.6921283
Cossine similarity to target: 0.5678471 

Original sentence: add remember to call mom to my reminders
Target sentence: aggiungi ricorda di chiamare mamma ai miei promemoria
Generated sentence: rimuovi spesa al supermercato dalla mia lista delle cose da fare domenica
Cossine similarity to generated: 0.85635823
Cossine similarity to target: 0.763936 

Original sentence: can you read me my lists
Target sentence: puoi leggermi le mie liste
Generated sentence: puoi cucinare qualcosa di speciale per me e mia moglie
Cossine similarity to generated: 0.8514999
Cossine similarity to target: 0.7323967 

Original sentence: hey what are my lists
Target sentence: ciao quali sono le mie liste
Generated sentence: dimmi quali sono i film migliori nella mia zona
Cossine similarity to generated: 0.83229375
Cossine similarity

,Cosine Similarity,Euclidean Distance
en -> it,0.707053,5.180112
en -> es -> it,0.698739,5.233174


## Saving results to spreadsheet

In [25]:
path = 'results.xlsx'
with pd.ExcelWriter(path) as writer:
    exp1.to_excel(writer, sheet_name = 'Study case 1')
    exp2.to_excel(writer, sheet_name = 'Study case 2')
    exp3.to_excel(writer, sheet_name = 'Study case 3')
    exp4.to_excel(writer, sheet_name = 'Study case 4')
    exp5.to_excel(writer, sheet_name = 'Experiment 1')
    exp6.to_excel(writer, sheet_name = 'Experiment 2')
    exp7.to_excel(writer, sheet_name = 'Experiment 3')
    exp8.to_excel(writer, sheet_name = 'Experiment 4')